In [11]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import transforms as T, models
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from tqdm.notebook import tqdm 

from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
import seaborn as sns
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
from random import shuffle, randint, choice
import numpy as np
import cv2 as cv
from glob import glob
import pandas as pd
import onnx
from onnx_tf.backend import prepare
from onnx import helper

In [15]:
full_df = pd.read_csv('trainLabels.csv')

full_df.head()

print(full_df['level'].value_counts())

# ### Undersampling Attempt
# print(full_df['level'].value_counts())
# nMax = 708
# # nMax = 50
# resampled_df = full_df.groupby('level').apply(lambda x: x.sample(n=min(nMax, len(x))))

# # print(resampled_df['level'].value_counts())

# # print(resampled_df.shape)

# class_0_df = resampled_df[resampled_df['level'] == 0]
# class_1_df = resampled_df[resampled_df['level'] == 1]
# class_2_df = resampled_df[resampled_df['level'] == 2]
# class_3_df = resampled_df[resampled_df['level'] == 3]
# class_4_df = resampled_df[resampled_df['level'] == 4]

# class_0_train, class_0_test = train_test_split(class_0_df, test_size=0.2, random_state=42)
# class_1_train, class_1_test = train_test_split(class_1_df, test_size=0.2, random_state=42)
# class_2_train, class_2_test = train_test_split(class_2_df, test_size=0.2, random_state=42)
# class_3_train, class_3_test = train_test_split(class_3_df, test_size=0.2, random_state=42)
# class_4_train, class_4_test = train_test_split(class_4_df, test_size=0.2, random_state=42)

# train_df = pd.concat([class_0_train, class_1_train, class_2_train, class_3_train, class_4_train], ignore_index=True)
# valid_df = pd.concat([class_0_test, class_1_test, class_2_test, class_3_test, class_4_test])

# print(train_df.head())
# print(train_df['level'].value_counts())

level
0    25810
2     5292
1     2443
3      873
4      708
Name: count, dtype: int64


In [13]:
# size_list = []
# for index, row in train_df.iterrows():
#     print(index)
#     print(row['image'])

#     filepath = "images_copy/images/" + row['image'] + ".jpeg"
        
#     image = cv.imread(filepath)
#     image = Image.fromarray(image)
#     image = Image.open(filepath)
#     width, height = image.size

#     new_width = height
#     new_height = height

#     left = (width - new_width) / 2
#     top = (height - new_height) / 2
#     right = (width + new_width) / 2
#     bottom = (height + new_height) / 2
    
#     image = image.crop((left, top, right, bottom))

#     print(type(image))
#     print(image.size)


# train_df['size'] = size_list

# train_df.head()

# print(train_df['size'].value_counts())

# Transformations for Class 1

Initial Number - 2443
End Result - 2443 * 11 = 24430

In [14]:
class_1_df = full_df[full_df['level'] == 1]
class_1_df.shape
class_1_df = class_1_df.reset_index(drop=True)
class_1_df.head()

test_class_1_df = pd.DataFrame()

for row in class_1_df.iterrows():
    print(row[1]['image'])
    image_name = row[1]['image']
    image_label = row[1]['level']
    
    
    filepath = 'images_augment/' + image_name + '.jpeg'
    save_folder = 'augmented_images/'

    dict = {'image':[], 'level':[]}
    

    #Image Cropping
    image = Image.open(filepath)
    width, height = image.size

    new_width = height
    new_height = height

    left = (width - new_width) / 2
    top = (height - new_height) / 2
    right = (width + new_width) / 2
    bottom = (height + new_height) / 2
    
    #Transform 1
    image = image.crop((left, top, right, bottom))

    #image.show()
    

    #2 - Horizontal Flipping
    horizontal_transformed_image = image.transpose(Image.FLIP_LEFT_RIGHT)

    #3 - Vertical Flipping
    vertical_transformed_image = image.transpose(Image.FLIP_TOP_BOTTOM)

    #4 - rotate 45
    rotated_image_1 = image.rotate(45)

    #5 - rotate 135
    rotated_image_2 = image.rotate(135)

    #6 - rotate 225
    rotated_image_3 = image.rotate(225)

    #7 - rotate 315
    rotated_image_4 = image.rotate(315)

    #8 - Gaussian Blurring
    blurred_image = image.filter(ImageFilter.GaussianBlur(radius = 5))

    #9 - Translating Images by 30
    # Define the translation values (in pixels)
    x_translation = 30  # Shift 50 pixels to the right

    # Translate the image
    translated_image_1 = image.copy()  # Create a copy of the input image
    translated_image_1 = translated_image_1.transform(
        image.size,
        Image.AFFINE,
        (1, 0, x_translation, 0, 1, 0)
    )

    #10 - Translating Images by 20
    # Define the translation values (in pixels)
    x_translation = 20  # Shift 50 pixels to the right

    # Translate the image
    translated_image_2 = image.copy()  # Create a copy of the input image
    translated_image_2 = translated_image_2.transform(
        image.size,
        Image.AFFINE,
        (1, 0, x_translation, 0, 1, 0)
    )

    #11 - Translating Images by 10
    # Define the translation values (in pixels)
    x_translation = 10  # Shift 50 pixels to the right

    # Translate the image
    translated_image_3 = image.copy()  # Create a copy of the input image
    translated_image_3 = translated_image_3.transform(
        image.size,
        Image.AFFINE,
        (1, 0, x_translation, 0, 1, 0)
    )

    transformed_images_list = [
        image,
        horizontal_transformed_image,
        vertical_transformed_image,
        rotated_image_1,
        rotated_image_2,
        rotated_image_3,
        rotated_image_4,
        blurred_image,
        translated_image_1,
        translated_image_2,
        translated_image_3
    ]

    for i in range(1, len(transformed_images_list) + 1):
        img = transformed_images_list[i-1]
        new_img_name = image_name + '_aug_' + str(i)
        saved_file_path = save_folder + new_img_name + '.jpeg'
        img.save(saved_file_path)

        dict['image'].append(new_img_name)
        dict['level'].append(image_label)
    
    temp_df = pd.DataFrame(dict)
    #print(temp_df.head())
    test_class_1_df = pd.concat([test_class_1_df, temp_df], ignore_index=True)


print(test_class_1_df.head())
print(test_class_1_df.shape)


15_left
17_right
30_left
36_left
114_left
114_right
178_left
178_right


KeyboardInterrupt: 

In [ ]:
print(test_class_1_df.head())
print(test_class_1_df.shape)

           image  level
0  15_left_aug_1      1
1  15_left_aug_2      1
2  15_left_aug_3      1
3  15_left_aug_4      1
4  15_left_aug_5      1
(26873, 2)


# Transformations for Class 2

In [17]:
class_2_df = full_df[full_df['level'] == 2]
class_2_df.shape
class_2_df = class_2_df.reset_index(drop=True)
class_2_df.head()

test_class_2_df = pd.DataFrame()

for row in class_2_df.iterrows():
    print(row[1]['image'])
    image_name = row[1]['image']
    image_label = row[1]['level']
    
    dict = {'image':[], 'level':[]}

    filepath = 'images_augment/' + image_name + '.jpeg'
    save_folder = 'augmented_images/'

    #Image Cropping
    image = Image.open(filepath)
    width, height = image.size

    new_width = height
    new_height = height

    left = (width - new_width) / 2
    top = (height - new_height) / 2
    right = (width + new_width) / 2
    bottom = (height + new_height) / 2
    
    image = image.crop((left, top, right, bottom))

    #image.show()

    #Horizontal Flipping
    horizontal_transformed_image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # horizontal_transformed_image.show()

    #Vertical Flipping
    vertical_transformed_image = image.transpose(Image.FLIP_TOP_BOTTOM)
    # vertical_transformed_image.show()

    #Rotating Images
    # See how first, seems like the edges of the eye circle get messed up for some images
    rotated_image = image.rotate(45)
    # rotated_image.show()

    #Gaussian Blurring
    blurred_image = image.filter(ImageFilter.GaussianBlur(radius = 5))
    # blurred_image.show()


    transformed_images_list = [
        image,
        horizontal_transformed_image,
        vertical_transformed_image,
        rotated_image,
        blurred_image
    ]
    
    for i in range(1, len(transformed_images_list) + 1):
        img = transformed_images_list[i-1]
        new_img_name = image_name + '_aug_' + str(i)
        saved_file_path = save_folder + new_img_name + '.jpeg'
        img.save(saved_file_path)

        dict['image'].append(new_img_name)
        dict['level'].append(image_label)
    
    
    temp_df = pd.DataFrame(dict)
    #print(temp_df.head())
    test_class_2_df = pd.concat([test_class_2_df, temp_df], ignore_index=True)


    # test_class_2_df.loc[len(test_class_2_df.index)] = [image_name + '_center_cropped', image_label]
    # test_class_2_df.loc[len(test_class_2_df.index)] = [image_name + '_horizontal_flipped', image_label]
    # test_class_2_df.loc[len(test_class_2_df.index)] = [image_name + '_vertical_flipped', image_label]
    # test_class_2_df.loc[len(test_class_2_df.index)] = [image_name + '_rotated_image_45', image_label]
    # test_class_2_df.loc[len(test_class_2_df.index)] = [image_name + '_gaussian_blurred', image_label]


print(test_class_2_df.head())
print(test_class_2_df.shape)


15_right
30_right
40_left
51_left
54_left
54_right
78_left
78_right
79_left
79_right
82_left
82_right
129_left
129_right
155_right
172_left
172_right
184_left
184_right
192_left
192_right
195_left
195_right
219_left
229_right
239_left
239_right
257_left
257_right
264_left
264_right
283_left
283_right
286_left
286_right
317_left
317_right
320_right
325_left
325_right
328_left
337_right
360_left
360_right
375_left
378_left
378_right
391_left
396_left
396_right
419_left
419_right
427_left
427_right
439_left
444_left
444_right
466_left
466_right
508_left
508_right
509_right
510_left
510_right
527_right
534_left
534_right
558_left
558_right
560_right
607_left
609_right
639_left
660_left
660_right
679_right
681_left
681_right
685_left
685_right
696_right
715_left
715_right
717_left
717_right
752_left
752_right
761_left
818_left
820_left
820_right
829_left
829_right
857_left
865_left
865_right
877_left
877_right
878_left
904_left
905_left
905_right
906_left
906_right
916_left
928_left
928_rig

In [ ]:
print(class_2_df.shape)

(5292, 2)


# Transformations for Class 3

In [18]:
class_3_df = full_df[full_df['level'] == 3]
class_3_df.shape
class_3_df = class_3_df.reset_index(drop=True)
print(class_3_df.head())

test_class_3_df = pd.DataFrame()

for row in class_3_df.iterrows():
    print(row[1]['image'])
    image_name = row[1]['image']
    image_label = row[1]['level']
    
    
    filepath = 'images_augment/' + image_name + '.jpeg'
    save_folder = 'augmented_images/'

    dict = {'image':[], 'level':[]}

    #Image Cropping
    image = Image.open(filepath)
    width, height = image.size

    new_width = height
    new_height = height

    left = (width - new_width) / 2
    top = (height - new_height) / 2
    right = (width + new_width) / 2
    bottom = (height + new_height) / 2
    
    #Transform 1
    image = image.crop((left, top, right, bottom))

    #image.show()
    
    #2 - Horizontal Flipping
    horizontal_transformed_image = image.transpose(Image.FLIP_LEFT_RIGHT)

    #3 - Vertical Flipping
    vertical_transformed_image = image.transpose(Image.FLIP_TOP_BOTTOM)

    #4 - Gaussian Blurring
    blurred_image = image.filter(ImageFilter.GaussianBlur(radius = 5))

    #4 - rotate 45
    rotated_image_1 = image.rotate(15)

    #4 - rotate 45
    rotated_image_2 = image.rotate(45)

    #5 - rotate 135
    rotated_image_3 = image.rotate(75)

    #6 - rotate 225
    rotated_image_4 = image.rotate(105)

    #7 - rotate 315
    rotated_image_5 = image.rotate(135)

    rotated_image_6 = image.rotate(165)

    rotated_image_7 = image.rotate(195)

    rotated_image_8 = image.rotate(225)

    rotated_image_9 = image.rotate(255)

    rotated_image_10 = image.rotate(285)

    rotated_image_11 = image.rotate(315)

    rotated_image_12 = image.rotate(345)

    #9 - Translating Images by 30
    # Define the translation values (in pixels)
    x_translation = 30  # Shift 50 pixels to the right

    # Translate the image
    translated_image_1 = image.copy()  # Create a copy of the input image
    translated_image_1 = translated_image_1.transform(
        image.size,
        Image.AFFINE,
        (1, 0, x_translation, 0, 1, 0)
    )

    #10 - Translating Images by 20
    # Define the translation values (in pixels)
    x_translation = 20  # Shift 50 pixels to the right

    # Translate the image
    translated_image_2 = image.copy()  # Create a copy of the input image
    translated_image_2 = translated_image_2.transform(
        image.size,
        Image.AFFINE,
        (1, 0, x_translation, 0, 1, 0)
    )

    #11 - Translating Images by 10
    # Define the translation values (in pixels)
    x_translation = 10  # Shift 50 pixels to the right

    # Translate the image
    translated_image_3 = image.copy()  # Create a copy of the input image
    translated_image_3 = translated_image_3.transform(
        image.size,
        Image.AFFINE,
        (1, 0, x_translation, 0, 1, 0)
    )

    #Transformation 20 - Rotate 45 + Gaussian Blur
    image_20 = rotated_image_2.copy()
    blurred_image_20 = image_20.filter(ImageFilter.GaussianBlur(radius = 5))

    #Transformation 21 - Rotate 135 + Gaussian Blur
    image_21 = rotated_image_5.copy()
    blurred_image_21 = image_21.filter(ImageFilter.GaussianBlur(radius = 5))

    #Transformation 22 - Rotate 225 + Gaussian Blur
    image_22 = rotated_image_8.copy()
    blurred_image_22 = image_22.filter(ImageFilter.GaussianBlur(radius = 5))

    #Transformation 23 - Translate 30 + Gaussian Blur
    image_23 = translated_image_1.copy()
    blurred_image_23 = image_23.filter(ImageFilter.GaussianBlur(radius=5))

    #Transformation 24 - Translate 20 + Gaussian Blur
    image_24 = translated_image_2.copy()
    blurred_image_24 = image_24.filter(ImageFilter.GaussianBlur(radius=5))

    #Transformation 25 - Translate 10 + Gaussian Blur
    image_25 = translated_image_3.copy()
    blurred_image_25 = image_25.filter(ImageFilter.GaussianBlur(radius=5))

    #Transformation 26 - VErtical Flip + Translate 30
    image_26 = vertical_transformed_image.copy()
    image_26 = image_26.transform(image_26.size, Image.AFFINE, (1, 0, 30, 0, 1, 0))

    #Transformation 27 - Horizontal Flip + Translate 30
    image_27 = horizontal_transformed_image.copy()
    image_27 = image_27.transform(image_27.size, Image.AFFINE, (1, 0, 30, 0, 1, 0))

    #Transformation 28 - vertical + translate 30 + gaussian
    image_28 = image_26.copy()
    image_28 = image_28.filter(ImageFilter.GaussianBlur(radius=5))

    #Transformation 29 - horizontal + translate 30 + gaussian
    image_29 = image_27.copy()
    image_29 = image_29.filter(ImageFilter.GaussianBlur(radius=5))

    transformed_images_list = [
        image,
        horizontal_transformed_image,
        vertical_transformed_image,
        blurred_image,
        rotated_image_1,
        rotated_image_2,
        rotated_image_3,
        rotated_image_4,
        rotated_image_5,
        rotated_image_6,
        rotated_image_7,
        rotated_image_8,
        rotated_image_9,
        rotated_image_10,
        rotated_image_11,
        rotated_image_12,
        translated_image_1,
        translated_image_2,
        translated_image_3,
        image_20,
        image_21,
        image_22,
        image_23,
        image_24,
        image_25,
        image_26,
        image_27,
        image_28,
        image_29
    ]

    for i in range(1, len(transformed_images_list) + 1):
        img = transformed_images_list[i-1]
        new_img_name = image_name + '_aug_' + str(i)
        saved_file_path = save_folder + new_img_name + '.jpeg'
        img.save(saved_file_path)

        dict['image'].append(new_img_name)
        dict['level'].append(image_label)
    
    temp_df = pd.DataFrame(dict)
    #print(temp_df.head())
    test_class_3_df = pd.concat([test_class_3_df, temp_df], ignore_index=True)

print(test_class_3_df.head())
print(test_class_3_df.shape)
    

       image  level
0    99_left      3
1   99_right      3
2   163_left      3
3  163_right      3
4  328_right      3
99_left
99_right
163_left
163_right
328_right
352_left
352_right
391_right
458_left
509_left
531_left
531_right
609_left
679_left
687_left
687_right
932_left
932_right
1002_left
1008_left
1008_right
1099_left
1162_left
1162_right
1177_left
1177_right
1196_left
1196_right
1350_right
1391_left
1420_left
1420_right
1471_left
1471_right
1639_left
1639_right
1640_left
1919_left
2016_right
2132_right
2275_left
2496_right
2532_left
2532_right
2577_left
2577_right
2767_left
2767_right
2862_left
2862_right
2901_right
3096_left
3096_right
3192_left
3287_left
3287_right
3307_left
3307_right
3317_right
3343_left
3343_right
3379_left
3379_right
3422_left
3422_right
3444_left
3444_right
3523_left
3570_left
3570_right
3748_right
3773_left
3773_right
3868_left
3868_right
3988_left
3992_left
3992_right
4148_right
4155_left
4282_left
4282_right
4366_left
4366_right
4374_right
4531_righ

In [19]:
class_4_df = full_df[full_df['level'] == 4]
class_4_df.shape
class_4_df = class_4_df.reset_index(drop=True)

test_class_4_df = pd.DataFrame()

for row in class_4_df.iterrows():
    print(row[1]['image'])
    image_name = row[1]['image']
    image_label = row[1]['level']
    
    
    filepath = 'images_augment/' + image_name + '.jpeg'
    save_folder = 'augmented_images/'

    dict = {'image':[], 'level':[]}

    #Image Cropping
    image = Image.open(filepath)
    width, height = image.size

    new_width = height
    new_height = height

    left = (width - new_width) / 2
    top = (height - new_height) / 2
    right = (width + new_width) / 2
    bottom = (height + new_height) / 2
    
    #Transform 1
    image = image.crop((left, top, right, bottom))

    #image.show()
    
    #2 - Horizontal Flipping
    horizontal_transformed_image = image.transpose(Image.FLIP_LEFT_RIGHT)

    #3 - Vertical Flipping
    vertical_transformed_image = image.transpose(Image.FLIP_TOP_BOTTOM)

    #4 - Gaussian Blurring
    blurred_image = image.filter(ImageFilter.GaussianBlur(radius = 5))

    #4 - rotate 45
    rotated_image_1 = image.rotate(15)

    #4 - rotate 45
    rotated_image_2 = image.rotate(45)

    #5 - rotate 135
    rotated_image_3 = image.rotate(75)

    #6 - rotate 225
    rotated_image_4 = image.rotate(105)

    #7 - rotate 315
    rotated_image_5 = image.rotate(135)

    rotated_image_6 = image.rotate(165)

    rotated_image_7 = image.rotate(195)

    rotated_image_8 = image.rotate(225)

    rotated_image_9 = image.rotate(255)

    rotated_image_10 = image.rotate(285)

    rotated_image_11 = image.rotate(315)

    rotated_image_12 = image.rotate(345)

    #9 - Translating Images by 30
    # Define the translation values (in pixels)
    x_translation = 30  # Shift 50 pixels to the right

    # Translate the image
    translated_image_1 = image.copy()  # Create a copy of the input image
    translated_image_1 = translated_image_1.transform(
        image.size,
        Image.AFFINE,
        (1, 0, x_translation, 0, 1, 0)
    )

    #10 - Translating Images by 20
    # Define the translation values (in pixels)
    x_translation = 20  # Shift 50 pixels to the right

    # Translate the image
    translated_image_2 = image.copy()  # Create a copy of the input image
    translated_image_2 = translated_image_2.transform(
        image.size,
        Image.AFFINE,
        (1, 0, x_translation, 0, 1, 0)
    )

    #11 - Translating Images by 10
    # Define the translation values (in pixels)
    x_translation = 10  # Shift 50 pixels to the right

    # Translate the image
    translated_image_3 = image.copy()  # Create a copy of the input image
    translated_image_3 = translated_image_3.transform(
        image.size,
        Image.AFFINE,
        (1, 0, x_translation, 0, 1, 0)
    )

    #Transformation 20 - Rotate 45 + Gaussian Blur
    image_20 = rotated_image_2.copy()
    blurred_image_20 = image_20.filter(ImageFilter.GaussianBlur(radius = 5))

    #Transformation 21 - Rotate 135 + Gaussian Blur
    image_21 = rotated_image_5.copy()
    blurred_image_21 = image_21.filter(ImageFilter.GaussianBlur(radius = 5))

    #Transformation 22 - Rotate 225 + Gaussian Blur
    image_22 = rotated_image_8.copy()
    blurred_image_22 = image_22.filter(ImageFilter.GaussianBlur(radius = 5))

    #Transformation 23 - Translate 30 + Gaussian Blur
    image_23 = translated_image_1.copy()
    blurred_image_23 = image_23.filter(ImageFilter.GaussianBlur(radius=5))

    #Transformation 24 - Translate 20 + Gaussian Blur
    image_24 = translated_image_2.copy()
    blurred_image_24 = image_24.filter(ImageFilter.GaussianBlur(radius=5))

    #Transformation 25 - Translate 10 + Gaussian Blur
    image_25 = translated_image_3.copy()
    blurred_image_25 = image_25.filter(ImageFilter.GaussianBlur(radius=5))

    #Transformation 26 - VErtical Flip + Translate 30
    image_26 = vertical_transformed_image.copy()
    image_26 = image_26.transform(image_26.size, Image.AFFINE, (1, 0, 30, 0, 1, 0))

    #Transformation 27 - Horizontal Flip + Translate 30
    image_27 = horizontal_transformed_image.copy()
    image_27 = image_27.transform(image_27.size, Image.AFFINE, (1, 0, 30, 0, 1, 0))

    #Transformation 28 - vertical + translate 30 + gaussian
    image_28 = image_26.copy()
    image_28 = image_28.filter(ImageFilter.GaussianBlur(radius=5))

    #Transformation 29 - horizontal + translate 30 + gaussian
    image_29 = image_27.copy()
    image_29 = image_29.filter(ImageFilter.GaussianBlur(radius=5))

    #Transformation 30 - rotate 45 + Translate 30
    image_30 = rotated_image_2.copy()
    image_30 = image_30.transform(image_30.size, Image.AFFINE, (1, 0, 30, 0, 1, 0))

    #Transformation 31 - rotate 135 + Translate 30
    image_31 = rotated_image_5.copy()
    image_31 = image_31.transform(image_31.size, Image.AFFINE, (1, 0, 30, 0, 1, 0))

    #Transformation 32 - rotate 225 + Translate 30
    image_32 = rotated_image_8.copy()
    image_32 = image_32.transform(image_32.size, Image.AFFINE, (1, 0, 30, 0, 1, 0))

    #Transformation 33 - rotate 315 + Translate 30
    image_33 = rotated_image_11.copy()
    image_33 = image_33.transform(image_33.size, Image.AFFINE, (1, 0, 30, 0, 1, 0))

    #Transformation 34 - rotate 45 + translate 30 + gaussian blur
    image_34 = image_30.copy()
    image_34 = image_34.filter(ImageFilter.GaussianBlur(radius = 5))

    #Transformation 35 - rotate 225 + translate 30 + gaussian blur
    image_35 = image_32.copy()
    image_35 = image_35.filter(ImageFilter.GaussianBlur(radius=5))

    #Transformation 36 - rotate 150 degrees
    image_36 = image.rotate(150)

    transformed_images_list = [
        image,
        horizontal_transformed_image,
        vertical_transformed_image,
        blurred_image,
        rotated_image_1,
        rotated_image_2,
        rotated_image_3,
        rotated_image_4,
        rotated_image_5,
        rotated_image_6,
        rotated_image_7,
        rotated_image_8,
        rotated_image_9,
        rotated_image_10,
        rotated_image_11,
        rotated_image_12,
        translated_image_1,
        translated_image_2,
        translated_image_3,
        image_20,
        image_21,
        image_22,
        image_23,
        image_24,
        image_25,
        image_26,
        image_27,
        image_28,
        image_29,
        image_30,
        image_31,
        image_32,
        image_33,
        image_34,
        image_35,
        image_36
    ]

    for i in range(1, len(transformed_images_list) + 1):
        img = transformed_images_list[i-1]
        new_img_name = image_name + '_aug_' + str(i)
        saved_file_path = save_folder + new_img_name + '.jpeg'
        img.save(saved_file_path)

        dict['image'].append(new_img_name)
        dict['level'].append(image_label)
    
    temp_df = pd.DataFrame(dict)
    #print(temp_df.head())
    test_class_4_df = pd.concat([test_class_4_df, temp_df], ignore_index=True)

print(test_class_4_df.head())
print(test_class_4_df.shape)
    

16_left
16_right
217_left
217_right
294_left
294_right
326_left
326_right
367_left
367_right
405_left
405_right
406_left
406_right
439_right
458_right
670_left
670_right
936_left
936_right
986_left
1084_left
1084_right
1099_right
1138_left
1138_right
1350_left
1430_left
1430_right
1663_left
1663_right
1941_left
2016_left
2031_right
2338_left
2338_right
2496_left
2804_left
2804_right
2810_left
2810_right
2880_right
2916_left
2916_right
2970_left
2970_right
3064_left
3064_right
3088_left
3371_left
3371_right
3412_left
3412_right
3418_left
3501_left
3563_left
3563_right
3611_right
3802_left
3802_right
3993_left
3993_right
4005_left
4005_right
4086_left
4086_right
4130_left
4130_right
4155_right
4252_left
4252_right
4374_left
4496_left
4496_right
4531_left
4688_right
4793_left
4909_left
4909_right
5032_left
5032_right
5105_left
5105_right
5258_left
5258_right
5304_right
5825_left
5825_right
5838_left
6017_left
6017_right
6096_left
6096_right
6613_left
6613_right
6707_left
6707_right
6719_l

In [10]:
class_0_df = full_df[full_df['level'] == 0]
class_0_df.shape

for row in class_0_df.iterrows():
    print(row[1]['image'])
    image_name = row[1]['image']
    image_label = row[1]['level']
    
    
    filepath = 'images_augment/' + image_name + '.jpeg'
    save_folder = 'augmented_images/'

    dict = {'image':[], 'level':[]}

    #Image Cropping
    image = Image.open(filepath)
    width, height = image.size

    new_width = height
    new_height = height

    left = (width - new_width) / 2
    top = (height - new_height) / 2
    right = (width + new_width) / 2
    bottom = (height + new_height) / 2
    
    #Transform 1
    image = image.crop((left, top, right, bottom))

    saved_file_path = save_folder + image_name + '.jpeg'
    image.save(saved_file_path)

10_left
10_right
13_left
13_right
17_left
19_left
19_right
20_left
20_right
21_left
21_right
22_left
22_right
23_left
23_right
25_left
25_right
31_left
31_right
33_left
33_right
36_right
40_right
41_left
41_right
42_left
42_right
46_left
46_right
47_left
47_right
49_left
49_right
51_right
52_left
52_right
56_left
56_right
57_left
57_right
58_left
58_right
59_left
59_right
60_left
60_right
62_left
62_right
64_left
64_right
65_left
65_right
66_left
66_right
67_left
67_right
70_left
70_right
72_left
72_right
73_left
73_right
74_left
74_right
75_left
75_right
81_left
81_right
84_left
84_right
95_left
95_right
100_left
100_right
102_left
102_right
104_left
104_right
108_left
108_right
111_left
111_right
115_left
115_right
117_left
117_right
119_left
119_right
122_left
122_right
125_left
125_right
140_left
140_right
145_left
145_right
149_left
149_right
155_left
156_left
156_right
157_left
157_right
159_left
159_right
161_left
161_right
162_left
162_right
164_left
164_right
167_left
167_righ

KeyboardInterrupt: 